Koda Maše Popovič

In [15]:
from sage.all import *

def metricna_dimenzija_usmerjenega_grafa(graf):
    
    if not isinstance(graf, DiGraph):
        return "Napaka: Podani graf ni usmerjen graf."
    
    # vozlišča grafa
    V = graf.vertices()
    
    # Izračun razdalj med vsemi pari vozlišč
    razdalje = {u: {v: graf.distance(u, v) for v in V} for u in V}
    
    # linearni program
    lp = MixedIntegerLinearProgram(maximization=False)
    x = lp.new_variable(binary=True)  # Ustvarjanje binarnih spremenljivk za vsako vozlišče
    
    # Cilj: minimizirati vsoto vseh x[v]
    lp.set_objective(sum(x[v] for v in V))
    
    # Preverjanje, ali graf izpolnjuje pogoje
    for u in V:
        for v in V:
            if u != v:
                vozlisca = [
                    w for w in V
                    if razdalje[w][u] != razdalje[w][v] and
                    razdalje[w][u] < infinity and
                    razdalje[w][v] < infinity
                ]
                
                # Če za par u, v ne obstaja ustrezno w, ne moremo izračunati metrične dimenzije
                if not vozlisca:
                    return f"Metrične dimenzije ni mogoče določiti: za par vozlišč ({u}, {v}) ne obstaja ustrezno vozlišče."
                
                # Dodamo omejitev, če obstajajo ustrezna vozlišča w
                lp.add_constraint(sum(x[w] for w in vozlisca) >= 1)
    
    # Rešitev linearnega programa
    lp.solve()
    
    # Pridobimo rezultate
    razresljiva_mnozica = [v for v in V if lp.get_values(x[v]) == 1]
    return razresljiva_mnozica, len(razresljiva_mnozica)


def clockwise_circulant_graph(n, d):
    odmiki = list(range(1, d + 1)) 
    G = digraphs.Circulant(n, odmiki)
    #plot = G.plot(layout="circular", vertex_size=300, vertex_color="skyblue", edge_color="black", 
                  #vertex_labels=True)
    #plot.show() #zanima naju samo dimenzija, zato sva odstranili del, ki nariše graf
    return G


Ali koda vrne pravilne rezultate? Preverimo na manjših grafih

In [2]:
rez = []
for n in range (4, 7):
    for d in range(2, 6):
        g = clockwise_circulant_graph(n, d)
        md = metricna_dimenzija_usmerjenega_grafa(g)
        rez.append(md)

for r in rez:
    print(r)

([0, 1], 2)
([0, 1, 3], 3)
([0, 1, 3], 3)
([0, 1, 3], 3)
([0, 1], 2)
([0, 2], 2)
([0, 1, 2, 3], 4)
([0, 1, 2, 3], 4)
([0, 5], 2)
([0, 1, 3], 3)
([0, 1, 3], 3)
([0, 1, 2, 4, 5], 5)


Kaj pa za večje grafe?

In [3]:
rez = []
for n in range (100, 103):
    for d in range(10, 11):
        g = clockwise_circulant_graph(n, d)
        md = metricna_dimenzija_usmerjenega_grafa(g)
        rez.append(md)

for r in rez:
    print(r)

([10, 39, 51, 92, 93, 94, 95, 96, 97, 98], 10)
([5, 52, 53, 54, 55, 61, 70, 79, 88, 97], 10)
([0, 6, 7, 15, 22, 24, 33, 41, 91, 100], 10)


Ali velja trditev $dim(C_n(1, \dots, k)) = k$ za $n > 2(k-1)^2$?

In [16]:
def preveri_dimenzijo(n, k):
    #funkcija ne deluje za k > n:
    if k >= n:
        return {"n": n, 
                "k": k, 
                "napaka": "Za cirkulantni graf mora veljati k < n."}
    
    # V primeru, da je n manjsi od funkcija f(k)
    if n <= 2*(k - 1)**2:
        return {"n": n, 
                "k": k, 
                "napaka": "Pogoj n > 2(k - 1)^2 ni izpolnjen."}

    # Ustvari cirkulantni graf
    Cn_k = clockwise_circulant_graph(n, k)

    # Izračuna razresljivo mn in pove metricno dimenzijo
    razresljiva_mnozica, dim = metricna_dimenzija_usmerjenega_grafa(Cn_k)

    # Preveri enakost
    rezultat = (dim == k)

    return {"n": n, 
            "k": k, 
            "dimCn_k": dim, 
            "trditev_velja": rezultat}

Različne vrednosti n za $k = 3$

In [5]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo(n, 3) for n in vrednosti_n]

#Zanka izpise vsak slovar v svoji vrstici
for r in rezultati:
    print(r)

{'n': 4, 'k': 3, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 5, 'k': 3, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 6, 'k': 3, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 7, 'k': 3, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 8, 'k': 3, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 9, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 10, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 11, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 12, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 13, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 14, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 15, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 16, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 17, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 18, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 19, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 20, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 21, 'k': 3

Različne vrednosti n za $k = 4$

In [6]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo(n, 4) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 4, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 6, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 7, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 8, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 9, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 10, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 11, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 12, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 13, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 14, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 15, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 16, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 17, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 18, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 19, '

Različne vrednosti za $k = 5$

In [7]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo(n, 5) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 5, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 5, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 6, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 7, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 8, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 9, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 10, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 11, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 12, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 13, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 14, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 15, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 16, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 17, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 18, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 

Različne vrednosti n za $k = 6$

In [8]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo(n, 6) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 6, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 7, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 8, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 9, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 10, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 11, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 12, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 13, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 14, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 15, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 16, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 17, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 18, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{

Vidimo, da trditev velja.

Ali bo $dim(C_n(1, \dots, k)) = k$ tudi za $n > 2(k-1)^2 - 1$?

In [17]:
def preveri_dimenzijo2(n, k):
    #funkcija ne deluje za k > n:
    if k >= n:
        return {"n": n, 
                "k": k, 
                "napaka": "Za cirkulantni graf mora veljati k < n."}
    
    # V primeru, da je n manjsi od funkcija f(k)
    if n <= 2*(k - 1)**2 - 1:
        return {"n": n, 
                "k": k, 
                "napaka": "Pogoj n > 2(k-1)^2 - 1 ni izpolnjen."}

    # Ustvari cirkulantni graf
    Cn_k = clockwise_circulant_graph(n, k)

    # Izračuna razresljivo mn in pove metricno dimenzijo
    razresljiva_mnozica, dim = metricna_dimenzija_usmerjenega_grafa(Cn_k)

    # Preveri enakost
    rezultat = (dim == k)

    return {"n": n, 
            "k": k, 
            "dimCn_k": dim, 
            "trditev_velja": rezultat}

Kaj se zgodi pri $k = 3$?

In [11]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo2(n, 3) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 3, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 5, 'k': 3, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 6, 'k': 3, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 7, 'k': 3, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 8, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 9, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 10, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 11, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 12, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 13, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 14, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 15, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 16, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 17, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 18, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 19, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 20, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 21, 'k': 3, '

Pri vrednosti $k = 4$:

In [7]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo2(n, 4) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 4, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 6, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 7, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 8, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 9, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 10, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 11, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 12, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 13, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 14, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 15, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 16, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 17, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 18, 'k': 4, 'dimCn_k': 4, 'trditev_velja': Tr

Pri vrednosti $k = 5$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo2(n, 5) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 5, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 5, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 6, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 7, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 8, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 9, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 10, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 11, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 12, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 13, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 14, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 15, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 16, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 17, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 18, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 

Pri vrednosti $k = 6$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo2(n, 6) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 6, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 7, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 8, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 9, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 10, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 11, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 12, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 13, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 14, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 15, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 16, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 17, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 18, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)

Opazimo, da je metrična dimenzija še vedno enaka $k$.

Ali bo $dim(C_n(1, \dots, k)) = k$ tudi za $n > 2(k-1)^2 - 2$?

In [18]:
def preveri_dimenzijo3(n, k):
    if k >= n:
        return {"n": n, 
                "k": k, 
                "napaka": "Za cirkulantni graf mora veljati k < n."}
    
    if n <= 2*(k - 1)**2 - 2:
        return {"n": n, 
                "k": k, 
                "napaka": "Pogoj n > 2(k-1)^2 - 2 ni izpolnjen."}

    Cn_k = clockwise_circulant_graph(n, k)
    razresljiva_mnozica, dim = metricna_dimenzija_usmerjenega_grafa(Cn_k)
    rezultat = (dim == k)

    return {"n": n, 
            "k": k, 
            "dimCn_k": dim, 
            "trditev_velja": rezultat}

Za $k = 3$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo3(n, 3) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 3, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 5, 'k': 3, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 6, 'k': 3, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 7, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 8, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 9, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 10, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 11, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 12, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 13, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 14, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 15, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 16, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 17, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 18, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 19, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 20, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 21, 'k': 3, 'dimCn_k': 3, 

Za $k = 4$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo3(n, 4) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 4, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 6, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 7, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 8, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 9, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 10, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 11, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 12, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 13, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 14, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 15, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 16, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 17, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 18, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 19,

Za $k = 5$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo3(n, 5) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 5, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 5, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 6, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 7, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 8, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 9, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 10, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 11, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 12, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 13, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 14, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 15, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 16, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 17, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 18, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 

Za $k = 6$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo3(n, 6) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 6, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 7, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 8, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 9, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 10, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 11, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 12, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 13, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 14, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 15, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 16, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 17, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 2 ni izpolnjen.'}
{'n': 18, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)

Tudi tu vidimo, da je metrična dimenzija grafa še vedno enaka $k$.

Kaj pa za $dim(C_n(1, \dots, k)) = k$ tudi za $n > 2(k-1)^2 - 5$? Ali sedaj pride do odstopanj?

In [19]:
def preveri_dimenzijo4(n, k):
    if k >= n:
        return {"n": n, 
                "k": k, 
                "napaka": "Za cirkulantni graf mora veljati k < n."}
    
    if n <= 2*(k - 1)**2 - 5:
        return {"n": n, 
                "k": k, 
                "napaka": "Pogoj n > 2(k-1)^2 - 5 ni izpolnjen."}

    Cn_k = clockwise_circulant_graph(n, k)
    razresljiva_mnozica, dim = metricna_dimenzija_usmerjenega_grafa(Cn_k)
    rezultat = (dim == k)

    return {"n": n, 
            "k": k, 
            "dimCn_k": dim, 
            "trditev_velja": rezultat}

Za $k = 3$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo4(n, 3) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 5, 'k': 3, 'dimCn_k': 2, 'trditev_velja': False}
{'n': 6, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 7, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 8, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 9, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 10, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 11, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 12, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 13, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 14, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 15, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 16, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 17, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 18, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 19, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 20, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 21, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 22, 'k': 

Za $k = 4$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo4(n, 4) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 4, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 6, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 7, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 8, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 9, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 10, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 11, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 12, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 13, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 14, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 15, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 16, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 17, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 18, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 19, 'k': 4, 'dimCn_k': 4, 'trditev_velja':

Za $k = 5$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo4(n, 5) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 5, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 5, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 6, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 7, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 8, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 9, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 10, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 11, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 12, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 13, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 14, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 15, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 16, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 17, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 18, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 

Za $k = 6$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo4(n, 6) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 6, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 7, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 8, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 9, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 10, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 11, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 12, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 13, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 14, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 15, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 16, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 17, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 18, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)

Opazimo, da metrična dimenzija grafa ne bo več enaka $k$ pri manjših $n$. Torej sva $f(k)$ tako zmanjšali,
da enakost ne velja več. Našli sva spodnjo mejo, pri kateri enakost ne velja več.

Sedaj si bova ogledali še vse $f(k)$ za katere velja: $2(k - 1)^2 - 5 < f(k) < 2(k - 1)^2$

Oglejmo si najprej $dim(C_n(1, \dots, k)) = k$ za $n > 2(k-1)^2 - 3$

In [20]:
def preveri_dimenzijo5(n, k):
    if k >= n:
        return {"n": n, 
                "k": k, 
                "napaka": "Za cirkulantni graf mora veljati k < n."}
    
    if n <= 2*(k - 1)**2 - 3:
        return {"n": n, 
                "k": k, 
                "napaka": "Pogoj n > 2(k-1)^2 - 3 ni izpolnjen."}

    Cn_k = clockwise_circulant_graph(n, k)
    razresljiva_mnozica, dim = metricna_dimenzija_usmerjenega_grafa(Cn_k)
    rezultat = (dim == k)

    return {"n": n, 
            "k": k, 
            "dimCn_k": dim, 
            "trditev_velja": rezultat}

Za $k = 3$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo5(n, 3) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 3, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 5, 'k': 3, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 6, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 7, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 8, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 9, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 10, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 11, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 12, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 13, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 14, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 15, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 16, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 17, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 18, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 19, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 20, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 21, 'k': 3, 'dimCn_k': 3, 'trditev_velj

Za $k = 4$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo5(n, 4) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 4, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 6, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 7, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 8, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 9, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 10, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 11, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 12, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 13, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 14, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 15, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 16, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 17, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 18, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 19, 'k': 4, 'dim

Za $k = 5$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo5(n, 5) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 5, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 5, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 6, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 7, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 8, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 9, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 10, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 11, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 12, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 13, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 14, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 15, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 16, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 17, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 18, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 

Za $k = 6$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo5(n, 6) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 6, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 7, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 8, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 9, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 10, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 11, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 12, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 13, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 14, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 15, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 16, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 17, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 3 ni izpolnjen.'}
{'n': 18, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)

Sedaj pa še $dim(C_n(1, \dots, k)) = k$ za $n > 2(k-1)^2 - 4$

In [21]:
def preveri_dimenzijo6(n, k):
    if k >= n:
        return {"n": n, 
                "k": k, 
                "napaka": "Za cirkulantni graf mora veljati k < n."}
    
    if n <= 2*(k - 1)**2 - 4:
        return {"n": n, 
                "k": k, 
                "napaka": "Pogoj n > 2(k-1)^2 - 4 ni izpolnjen."}

    Cn_k = clockwise_circulant_graph(n, k)
    razresljiva_mnozica, dim = metricna_dimenzija_usmerjenega_grafa(Cn_k)
    rezultat = (dim == k)

    return {"n": n, 
            "k": k, 
            "dimCn_k": dim, 
            "trditev_velja": rezultat}

Za $k = 3$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dimenzijo6(n, 3) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 3, 'napaka': 'Pogoj n > 2(k-1)^2 - 4 ni izpolnjen.'}
{'n': 5, 'k': 3, 'dimCn_k': 2, 'trditev_velja': False}
{'n': 6, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 7, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 8, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 9, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 10, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 11, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 12, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 13, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 14, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 15, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 16, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 17, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 18, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 19, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 20, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 21, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'

Opazimo, da metrična dimenzija grafa pri $n = 3$, ni enaka 3, torej smo našli protiprimer.

Kaj se zgodi, če uzamemo $f(k) = 2(k-1)$?


In [22]:
def preveri_dim_lin(n, k):
    if k >= n:
        return {"n": n, 
                "k": k, 
                "napaka": "Za cirkulantni graf mora veljati k < n."}
    
    if n <= 2*(k - 1):
        return {"n": n, 
                "k": k, 
                "napaka": "Pogoj n > 2(k - 1) ni izpolnjen."}

    Cn_k = clockwise_circulant_graph(n, k)
    razresljiva_mnozica, dim = metricna_dimenzija_usmerjenega_grafa(Cn_k)
    rezultat = (dim == k)

    return {"n": n, 
            "k": k, 
            "dimCn_k": dim, 
            "trditev_velja": rezultat}

Za $k = 3$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dim_lin(n, 3) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 3, 'napaka': 'Pogoj n > 2(k - 1) ni izpolnjen.'}
{'n': 5, 'k': 3, 'dimCn_k': 2, 'trditev_velja': False}
{'n': 6, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 7, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 8, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 9, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 10, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 11, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 12, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 13, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 14, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 15, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 16, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 17, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 18, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 19, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 20, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 21, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 

Za $k = 4$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dim_lin(n, 4) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 4, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1) ni izpolnjen.'}
{'n': 6, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1) ni izpolnjen.'}
{'n': 7, 'k': 4, 'dimCn_k': 3, 'trditev_velja': False}
{'n': 8, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 9, 'k': 4, 'dimCn_k': 3, 'trditev_velja': False}
{'n': 10, 'k': 4, 'dimCn_k': 3, 'trditev_velja': False}
{'n': 11, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 12, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 13, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 14, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 15, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 16, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 17, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 18, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 19, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 20, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 21, 'k': 4, 'dimCn_k': 4, 't

Za $k = 5$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dim_lin(n, 5) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 5, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 5, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 6, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1) ni izpolnjen.'}
{'n': 7, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1) ni izpolnjen.'}
{'n': 8, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1) ni izpolnjen.'}
{'n': 9, 'k': 5, 'dimCn_k': 4, 'trditev_velja': False}
{'n': 10, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 11, 'k': 5, 'dimCn_k': 4, 'trditev_velja': False}
{'n': 12, 'k': 5, 'dimCn_k': 3, 'trditev_velja': False}
{'n': 13, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 14, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 15, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 16, 'k': 5, 'dimCn_k': 4, 'trditev_velja': False}
{'n': 17, 'k': 5, 'dimCn_k': 4, 'trditev_velja': False}
{'n': 18, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 19, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 20, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 2

Za $k = 6$

In [ ]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
               21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
               36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
               51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
rezultati = [preveri_dim_lin(n, 6) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 6, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 7, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1) ni izpolnjen.'}
{'n': 8, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1) ni izpolnjen.'}
{'n': 9, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1) ni izpolnjen.'}
{'n': 10, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1) ni izpolnjen.'}
{'n': 11, 'k': 6, 'dimCn_k': 5, 'trditev_velja': False}
{'n': 12, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 13, 'k': 6, 'dimCn_k': 5, 'trditev_velja': False}
{'n': 14, 'k': 6, 'dimCn_k': 4, 'trditev_velja': False}
{'n': 15, 'k': 6, 'dimCn_k': 5, 'trditev_velja': False}
{'n': 16, 'k': 6, 'dimCn_k': 5, 'trditev_velja': False}
{'n': 17, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 18, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 19, 'k': 6, 'dimCn_k': 5, 'trditev_velja': False}
{'n': 20, 'k': 6, 'dimCn_k': 4, 'trd

Excel tabela z zbranimi rezultati

In [24]:
import pandas as pd

vrednosti_n = list(range(4, 78))

#f(k) = 2(k-1)^2
rezultati0 = [preveri_dimenzijo(n, k) for k in (3,4,5,6) for n in vrednosti_n]
df0 = pd.DataFrame(rezultati0)
pivot0 = df0.pivot(index="n", columns="k", values="dimCn_k")

#f(k) = 2(k-1)^2 - 1
rezultati1 = [preveri_dimenzijo2(n, k) for k in (3,4,5,6) for n in vrednosti_n]
df1 = pd.DataFrame(rezultati1)
pivot1 = df1.pivot(index="n", columns="k", values="dimCn_k")

#f(k) = 2(k-1)^2 - 2
rezultati2 = [preveri_dimenzijo3(n, k) for k in (3,4,5,6) for n in vrednosti_n]
df2 = pd.DataFrame(rezultati2)
pivot2 = df2.pivot(index="n", columns="k", values="dimCn_k")

#f(k) = 2(k-1)^2 - 3
rezultati3 = [preveri_dimenzijo5(n, k) for k in (3,4,5,6) for n in vrednosti_n]
df3 = pd.DataFrame(rezultati3)
pivot3 = df3.pivot(index="n", columns="k", values="dimCn_k")

#f(k) = 2(k-1)^2 - 4
rezultati4 = [preveri_dimenzijo6(n, k) for k in (3,4,5,6) for n in vrednosti_n]
df4 = pd.DataFrame(rezultati4)
pivot4 = df4.pivot(index="n", columns="k", values="dimCn_k")

#f(k) = 2(k-1)^2 - 5
rezultati5 = [preveri_dimenzijo4(n, k) for k in (3,4,5,6) for n in vrednosti_n]
df5 = pd.DataFrame(rezultati5)
pivot5 = df5.pivot(index="n", columns="k", values="dimCn_k")    

#f(k) = 2(k-1)
rezultati6 = [preveri_dim_lin(n, k) for k in (3,4,5,6) for n in vrednosti_n]
df6 = pd.DataFrame(rezultati6)
pivot6 = df6.pivot(index="n", columns="k", values="dimCn_k")


with pd.ExcelWriter("rezultati_dimenzija_cirkulantnih_grafov.xlsx") as writer:
    pivot0.to_excel(writer, sheet_name="f(k)=2(k-1)^2")
    pivot1.to_excel(writer, sheet_name="f(k)=2(k-1)^2-1")
    pivot2.to_excel(writer, sheet_name="f(k)=2(k-1)^2-2")
    pivot3.to_excel(writer, sheet_name="f(k)=2(k-1)^2-3")
    pivot4.to_excel(writer, sheet_name="f(k)=2(k-1)^2-4")
    pivot5.to_excel(writer, sheet_name="f(k)=2(k-1)^2-5")  
    pivot6.to_excel(writer, sheet_name="f(k)=2(k-1)") 

Preveri, ali mora ta celica nujno ostati (ali jo rabim za nadalnjo poganjanje kode)

In [ ]:
import sys
!{sys.executable} -m pip install pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 9.4 MB/s  0:00:016m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [openpyxl]2/3 [openpyxl]
